<a href="https://colab.research.google.com/github/DaniyalK03/Options-Calculator/blob/main/Option_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
################################################################################

# Importing standard backend and libraries

#Import standard libraries and imageio to read in the images
import imageio.v3 as imageio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

import math

# Importing TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

In [ ]:
from scipy.stats import norm
import numpy as np

def BS_option(S0, K, T, r, sigma, option_type = ""):
  """

  This function prices European Call & Put options using the Black-Scholes Model given

  S0 = Price of the stock today
  K = Strike Price
  T = Time to Maturity in Years
  r = Risk-free interest rate as Percent
  sigma = Volatility

  The result is the price of the option today

  """
  r = r/100
  d1 = (np.log(S0/K) + (r + (sigma**2)/2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - (sigma * np.sqrt(T))
  if option_type == "Call":
    return print(f"Call Price = {S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2):0.2f}")
  elif option_type == "Put":
    return print(f"Put Price = {K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1):0.2f}")
  else:
    print("ERROR, Please ensure you have specified an option with proper capitalisation")

In [ ]:
#BS_option(100, 100, 1, 8, 0.1, "Call")
#BS_option(100, 100, 1, 8, 0.1, "Put")
#BS_option(17.99, 15, 1, 5, 0.012, "Call")
BS_option(116.37, 130, 1, 5, 0.1, "Call")
BS_option(116.37, 130, 1, 5, 0.1, "Put")

Call Price = 2.00
Put Price = 9.29


In [ ]:
def Arbitrage(S0, K, T, r, call_price, put_price):

  call_price = float(call_price)
  put_price = float(put_price)

  r = r/100
  r = float(r)
  S0 = float(S0)
  K = float(K)
  T = float(T)

  lhs = put_price + S0
  rhs = call_price + K*np.exp(-r*T)

  avg_value = (lhs + rhs) / 2
  tolerance = 1e-5 * avg_value

  print(f"Tolerance: {tolerance} \n")


  if lhs < rhs - tolerance:
    arbitrage_profit = rhs - lhs

    # Show profit with precision based on the arbitrage size
    if round(arbitrage_profit, 2) > 0:
      print(f"Put-Call Parity Theorem states Put is underpriced relative to the Call.")
      print(f"Final Arbitrage Profit = {arbitrage_profit:0.2f}")
      print(f"\n Buy Put & Buy Stock \n Sell Call \n Invest cash difference into bank account to earn compound interest. \n Exercise option at {T} years. \n")
    else:
      print(f"Put-Call Parity Theorem states the options are priced fairly and there is no arbitrage.\n")


  elif lhs > rhs + tolerance:
    arbitrage_profit = lhs - rhs

    # Show profit with precision based on the arbitrage size
    if round(arbitrage_profit, 2) > 0:
      print(f"Put-Call Parity Theorem states Put is overpriced relative to the Call.")
      print(f"Final Arbitrage Profit = {arbitrage_profit:0.2f}")
      print(f"\n Sell Put & Sell Stock \n Buy Call \n Invest cash difference into bank account to earn compound interest. \n Exercise option at {T} years. \n")
    else:
      print(f"Put-Call Parity Theorem states the options are priced fairly and there is no arbitrage.\n")


  else:
    print("Put-Call Parity Theorem states the options are priced fairly and there is no arbitrage.\n")

In [ ]:
#Arbitrage(100, 100, 1, 8, 8.84, 1.15)
#Arbitrage(100, 100, 1, 8, 10.50, 0.50)
Arbitrage(100, 100, 1, 8, 7.00, 4.50)
Arbitrage(116.37, 130, 1, 5, 2, 9.29)

Tolerance: 0.001019058173193318 

Put-Call Parity Theorem states Put is overpriced relative to the Call.
Final Arbitrage Profit = 5.19

 Sell Put & Sell Stock 
 Buy Call 
 Invest cash difference into bank account to earn compound interest. 
 Exercise option at 1.0 years. 

Tolerance: 0.0012565991259254642 

Put-Call Parity Theorem states the options are priced fairly and there is no arbitrage.



In [ ]:
def binomial_tree(S, K, T, r, sigma, n=100, option_type='call'):
    dt = T/n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1/u
    p = (np.exp(r * dt) - d) / (u - d)
    # Initialize stock price tree
    stock_tree = np.zeros((n+1, n+1))
    option_tree = np.zeros((n+1, n+1))

    # Stock prices at maturity
    for i in range(n+1):
        stock_tree[i, n] = S * (u ** (n-i)) * (d ** i)

    # Option payoffs at maturity
    if option_type == 'call':
        option_tree[:, n] = np.maximum(0, stock_tree[:, n] - K)
    else:
        option_tree[:, n] = np.maximum(0, K - stock_tree[:, n])

    # Backward induction
    for j in range(n-1, -1, -1):
        for i in range(j+1):
            option_tree[i, j] = np.exp(-r * dt) * (p * option_tree[i, j+1] + (1-p) * option_tree[i+1, j+1])
    return option_tree[0, 0]